In [78]:
from pathlib import Path
from rapidfuzz.fuzz import ratio
from rapidfuzz import process, fuzz
import pandas as pd
import re

csv_file_path =  "schrute.csv"
df = pd.read_csv(csv_file_path, header=0)
df['lower'] = df['text'].str.lower()
df['lower'] = df['lower'].str.strip()

In [88]:
def partial_ratio_word_level(str1, str2, **kwargs):
    words1 = str1.split()
    words2 = str2.split()
    len1 = len(words1)
    len2 = len(words2)
    if len1 > len2:
        return 0.0
    best_score = 0.0
    max_start = len2 - len1 + 1
    ratio = fuzz.ratio
    for i in range(max_start):
        substring_words = words2[i:i+len1]
        score = ratio(" ".join(words1), " ".join(substring_words))
        if score > best_score:
            best_score = score
    return best_score

# Example usage
string1 = "do you also get two week"

string2 = "it's going to be super weird, he just fired you last week."
string2 = "Okay, Michael handed in his 2-week notice, did you also hand in your 2-week?"
# string2 = "do you want to get some breakfast or something?"
score = partial_ratio_word_level(string1, string2)
print(f"Partial Ratio Score: {score}%")  # Output: 100.00%

Partial Ratio Score: 60.37735849056604%


In [24]:
def fast_scorer(a, b, **kwargs):
    # Ignore comparisons if b is shorter than a
    if len(b) < len(a):
        return 0  # Return a score of 0 if b can't be a match

    # Otherwise, use the default partial ratio for speed
    return fuzz.partial_ratio(a, b)

In [90]:
# Example data
query = "do you also hand two week"
# Perform matching
fuzz_match = process.extract(
        query, choices=df['lower'], scorer=partial_ratio_word_level, limit=5)
fuzz_match

[('okay, michael handed in his 2-week notice, did you also hand in your 2-week?',
  74.07407407407408,
  24163),
 ("why do you always do that? whenever i'm getting married, you don't believe me.",
  67.85714285714286,
  20169),
 ('why do you need to wear the holster at all?', 66.66666666666667, 40712),
 ('do you want to see a video of a weatherman who says "bold front" instead of "cold front"? it\'s insane.',
  66.66666666666667,
  54291),
 ('no, no. um, you always want to keep a sweater or cardigan of some sort, in case it gets drafty.',
  65.38461538461539,
  12297)]